In [1]:
import numpy as np
from numpy.random import default_rng
import matplotlib as mpl
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import AdaBoostClassifier
from sklearn.neural_network import MLPClassifier
from sklearn.datasets import fetch_openml
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import zero_one_loss
from sklearn.metrics import mean_squared_error
from tqdm import tqdm
np.random.seed(42)

In [3]:
def make_dataset():
    rng = default_rng()
    X, y = fetch_openml('mnist_784', return_X_y=True, as_frame=False)
    p = rng.permutation(X.shape[0])
    X = X[p]
    y = y[p]
    X = X.reshape((X.shape[0], -1))

    Xt, Xv, y_base, yv = train_test_split(X, y, test_size=0.2)

    scaler = StandardScaler()
    Xt = scaler.fit_transform(Xt)
    Xv = scaler.transform(Xv)


    Y = y_base
    num_data = len(y_base)
    noise_percent = []
    """
    for rr in np.linspace(0, num_data*0.3, 4, dtype='int'):
        noise_percent.append(rr / num_data)
        rand_idx = np.random.choice(num_data, size=(rr,), replace=False)
        print(rand_idx.shape)
        noisy_y = y_base
        noisy_y[rand_idx] = ~noisy_y[rand_idx]
        Y = np.vstack((Y, noisy_y))
    """
    return Xt, Xv, Y, yv, noise_percent
    
Xt, Xv, y, yv, noise_percent = make_dataset()

In [ ]:
y = y.astype('int')
yv = yv.astype('int')
plt.subplot(211)
trees = [10]
leafs = np.linspace(10**1, 10**2, 20, dtype='int')
zotrain_scores = []
zotest_scores = []
sqtrain_scores = []
sqtest_scores = []
for tn in trees:
    for ln in tqdm(leafs):
        clf = AdaBoostClassifier(n_estimators=ln)
        clf.fit(Xt[:10**3], y[:10**3])
        zotrain_scores.append(zero_one_loss(y, clf.predict(Xt)))
        zotest_scores.append(zero_one_loss(yv, clf.predict(Xv)))
        #sqtrain_scores.append(mean_squared_error(y[:10**4], clf.predict(Xt[:10**4])))
        #sqtest_scores.append(mean_squared_error(yv, clf.predict(Xv)))
plt.plot(leafs, zotrain_scores, label="Zero/One Training Scores")
plt.plot(leafs, zotest_scores, label="Zero/One Test Scores")
#plt.plot(np.log10(leafs), sqtrain_scores, label="MSE Training Scores")
#plt.plot(np.log10(leafs), sqtest_scores, label="MSE Test Scores")
plt.legend(bbox_to_anchor=(1,1), loc="upper left")
plt.show()

 95%|█████████▌| 19/20 [02:35<00:13, 13.14s/it]